## 노드5 지도학습(분류)

- 분류모델
    - 의사결정나무
    - 랜덤포레스트
    - xgboost
- 교차검증
- 평가(분류)

In [1]:
# 라이브러리 불러오기
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## 데이터 불러오기

In [2]:
# 데이터 생성
from sklearn.datasets import load_breast_cancer

def make_dataset():
    iris = load_breast_cancer()
    df = d=pd.DataFrame(iris.data, columns=iris.feature_names)
    df['target'] = iris.target
    X_train, X_test, y_train, y_test = train_test_split(
        df.drop('target', axis=1), df['target'], test_size=0.5, random_state=1004)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = make_dataset()
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((284, 30), (285, 30), (284,), (285,))

In [3]:
y_train.value_counts()

1    190
0     94
Name: target, dtype: int64

## 의사결정나무
- 지도학습(분류)에서 가장 유용하게 사용되고 있는 기법 중 하나임
- 트리의 루트(root)에서 시작해서 정보이득이 최대가 되는 특성으로 데이터를 나눔
- 정보이득(Infomation gain)이 최대가 되는 특성을 나누는 기준(불순도를 측정하는 기준)은 '지니'와 '엔트로피'가 사용됨
- 데이터가 한 종류만 있다면 엔트로피/지니 불순도는 0에 가까움, 서로 다른 데이터의 비율이 비슷하면 1에 가까움
- 정보이득(Infomation gain)이 최대라는 것은 붙순도를 최소화 하는 방향임 (1-불순도)

In [4]:
# 의사결정나무
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred)

0.9263157894736842

## 의사결정나무 하이퍼파라미터
- criterion(기본값 gini) : 불순도 지표 (또는 엔트로피 불순도 entropy)
- max_depth(기본값 Nane): 최대 한도 깊이
- max leaf_ modes(기본값 None): 리프 노드의 최대 개수
- min_samples split(기본값 2): 자식 노드들 갖기 위한 최소한의 데이터 수
- min_samples leatf(기본값 1): 리프 노드가 되기 위한 최소 샘플 수

In [9]:
# 의사결정나무
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(
    criterion = 'entropy', 
    max_depth = 5,
    min_samples_split=6,
    min_samples_leaf=2,
    random_state=0)
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred)

0.9403508771929825

## 랜덤포레스트

- 여러개의 의사결정 트리로 구성
- 앙상블 방법 중 배깅(BAGGING) 방식
- 부트스트랩 샘플링 (데이터셋 중복 허용)
- 최종 다수결 투표

In [11]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=0)
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred)

0.9438596491228071

## 랜덤포레스트 하이퍼파라미터
- n_estimators[](기본값 100): 트리의 수
- criterion(기본값 gini): 불순도 지표
- max depth(기본값 None): 최대 한도 깊이
- min_samples_split(기본값 2): 자식 노드를 갖기 위한 최소한의 데이터 수
- min_samples_leaf(기본값 1): 리프 노드가 되기 위한 최소 샘플 수

In [15]:
# 랜덤포레스트 하이퍼파라미터
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=500, max_depth=5, random_state=0)
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred)

# 0.9473684210526315

0.9473684210526315

## xgboost

- 부스팅(앙상블) 기반의 알고리즘
- 캐글(글로벌 AI 경진대회) 뛰어난 성능을 보이면서 인기가 높아짐

In [20]:
# xgboost
from xgboost import XGBClassifier
model = XGBClassifier(random_state=0, use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred)

0.9508771929824561

### xgboost 하이퍼파라미터
- booster(기본값 gbtree): 부스팅 알고리즘 (또는 dart, gblinear)
- objective(기본값 binary:logistic): 이진분류 (다중분류: multi:softmax)
- max_depth(기본값 6): 최대 한도 깊이
- learning_rate(기본값 0.1): 학습률
- n_estimators(기본값 100): 트리의 수
- subsample(기본값 1): 훈련 샘플 개수의 비율
- colsample_bytree(기본값 1): 특성 개수의 비율
- n_jobs(기본값 1): 사용 코어 수 (-1: 모든 코어를 다 사용)

In [26]:
# xgboost
from xgboost import XGBClassifier
model = XGBClassifier(random_state=0, use_label_encoder=False, eval_metric='logloss',
                      booster = 'gbtree',
                      objective = 'binary:logistic',
                      max_depth = 5,
                      learning_rate = 0.05,
                      n_estimators = 700,
                      subsample = 1, 
                      colsample_bytree = 1,
                      n_jobs = -1)
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred)

0.9649122807017544

In [27]:
# 조기종료
model = XGBClassifier(random_state=0, use_label_encoder=False, eval_metric='logloss', 
                     learning_rate = 0.05,
                      n_estimators = 500)
eval_set = [(X_test, y_test)]
model.fit(X_train, y_train, eval_set=eval_set, early_stopping_rounds=10)
pred = model.predict(X_test)
accuracy_score(y_test, pred)

[0]	validation_0-logloss:0.65391
[1]	validation_0-logloss:0.61861
[2]	validation_0-logloss:0.58697
[3]	validation_0-logloss:0.55756
[4]	validation_0-logloss:0.53038
[5]	validation_0-logloss:0.50611
[6]	validation_0-logloss:0.48363
[7]	validation_0-logloss:0.46304
[8]	validation_0-logloss:0.44332
[9]	validation_0-logloss:0.42512
[10]	validation_0-logloss:0.40821
[11]	validation_0-logloss:0.39260
[12]	validation_0-logloss:0.37838
[13]	validation_0-logloss:0.36512
[14]	validation_0-logloss:0.35276
[15]	validation_0-logloss:0.34090
[16]	validation_0-logloss:0.33018
[17]	validation_0-logloss:0.31967
[18]	validation_0-logloss:0.30998
[19]	validation_0-logloss:0.30105
[20]	validation_0-logloss:0.29259
[21]	validation_0-logloss:0.28478
[22]	validation_0-logloss:0.27725
[23]	validation_0-logloss:0.27027
[24]	validation_0-logloss:0.26359
[25]	validation_0-logloss:0.25755
[26]	validation_0-logloss:0.25139
[27]	validation_0-logloss:0.24593
[28]	validation_0-logloss:0.24103
[29]	validation_0-loglos

0.9473684210526315

## 교차검증

In [29]:
# 데이터셋 로드
def make_dataset2():
    bc = load_breast_cancer()
    df = pd.DataFrame(bc.data, columns=bc.feature_names)
    df['target'] = bc.target
    return df.drop('target', axis=1), df['target']
X, y = make_dataset2()

Kfold
- 일반적으로 사용되는 교차검증 방법

In [31]:
from sklearn.model_selection import KFold
model = DecisionTreeClassifier(random_state=0)

kfold = KFold(n_splits=5)
for train_idx, test_idx in kfold.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    print(accuracy_score(y_test, pred))
    

0.8771929824561403
0.9122807017543859
0.9473684210526315
0.9385964912280702
0.8407079646017699


StratofiedKfold
- 불균형한 타겟 비율을 가진 데이터가 한쪽으로 치우치는 것을 방지

In [32]:
from sklearn.model_selection import StratifiedKFold
model = DecisionTreeClassifier(random_state=0)

kfold = StratifiedKFold(n_splits=5)
for train_idx, test_idx in kfold.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    print(accuracy_score(y_test, pred))

0.9035087719298246
0.9210526315789473
0.9122807017543859
0.9473684210526315
0.9026548672566371


### 사이킷런 교차검증
- 내부 API를 통해 fit - predict - evaluation

In [34]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, y, cv=3)
scores

array([0.88947368, 0.94210526, 0.86243386])

In [35]:
# 평균 점수
scores.mean()

0.8980042699340944

In [36]:
# 교차 검증 StratifiedKFold
kfold = StratifiedKFold(n_splits=5)
scores = cross_val_score(model, X, y, cv=kfold)
scores

array([0.90350877, 0.92105263, 0.9122807 , 0.94736842, 0.90265487])

In [37]:
# 평균 점수
scores.mean()

0.9173730787144851

## 평가(분류)

- **정확도 Accuracy**: 실젯값과 예측값이 일치 비율
- **정밀도 precision**: 양성이라고 예측한 값 중 실제 양성인 값 (암이라고 예측 한 값 중 실제 암일 확률)
- 재현율 recall: 실제 양성 값 중 양성으로 예측한 값 (실제 암인 값 중 예측도 암인 확률)
- F1: 정밀도와 재현율 조화 평균
- ROC-AUC
    - ROC: 참 양성 비율(True Positive Rate)에 대한 거짓 양성 비율(False Positive Rate) 곡선
    - AUC: ROC곡선 면적 아래 (완벽하게 분류되면 AUC가 1임)

In [38]:
# 정확도
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

0.9026548672566371

In [39]:
# 정밀도
from sklearn.metrics import precision_score
precision_score(y_test, pred)

0.9545454545454546

In [40]:
# 재현율
from sklearn.metrics import recall_score
recall_score(y_test, pred)

0.8873239436619719

In [41]:
# f1
from sklearn.metrics import f1_score
f1_score(y_test, pred)

0.9197080291970803

In [46]:
# roc_auc
from sklearn.metrics import roc_auc_score

model = XGBClassifier(random_state=0, use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)
pred = model.predict_proba(X_test)
roc_auc_score(y_test, pred[:,1])

0.999664654594232